### Logistic Regression Credit Risk

Dataset generado de forma aleatoria para efectos educativos.

### Datos de contacto

- Correo electronico: ortizmontilla@gmail.com


In [ ]:
''' 
pip install xgboost scikit-learn
pip install graphviz xgboost
'''

In [1]:
pwd

'/Users/michael/Documents/Python/ProyestosPersonales/Modelos/statistical_models/xgboots/01 - Credit_risk_model/NOTEBOOKS'

In [2]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, auc

# Gráficos
import matplotlib.pyplot as plt

In [3]:
# Leer el archivo CSV generado en un DataFrame
data = pd.read_csv('/Users/michael/Documents/Python/ProyestosPersonales/Modelos/statistical_models/xgboots/01 - Credit_risk_model/DATA/dataset_crediticio.csv',sep=';')
# Transformar los nombres de las columnas a minúsculas
data.columns = [column.lower() for column in data.columns]
data.shape

(10000, 16)

In [4]:
data.head(3)

,id,edad,ingresos_anuales,historial_crediticio,monto_del_prestamo,plazo_del_prestamo,tasa_de_interes,deuda_existente,numero_de_dependientes,estado_laboral,proposito_del_prestamo,estado_civil,nivel_de_educacion,score_de_riesgo_interno,resultado_del_riesgo,periodo
0,1,31,"$78,719.00",Bueno,"$35,098.00",30,11.8%,"$10,438.00",3,Autónomo,Otros,Divorciado,Maestría,507,1,2022-06
1,2,29,"$43,660.00",Regular,"$43,304.00",34,7.4%,"$1,581.00",0,Desempleado,Emergencia,Divorciado,Maestría,804,1,2022-05
2,3,58,"$56,278.00",Malo,"$12,629.00",9,10.9%,"$24,983.00",5,Desempleado,Auto,Soltero,Preparatoria,629,1,2022-04


In [5]:
# Tasa de malos
bad_rate = data['resultado_del_riesgo'].value_counts().reset_index()
bad_rate = bad_rate.rename(columns={'index': 'Categorias', 'resultado_del_riesgo': 'Qtd'})
bad_rate['%'] = round(bad_rate['Qtd'] / bad_rate['Qtd'].sum(), 1) * 100
bad_rate

,Categorias,Qtd,%
0,1,8892,90.0
1,0,1108,10.0


### Transformación en WoEs

In [6]:
# Definir las categorías y calcular el WOE para una variables independientes
def calcular_woe_variable(variable, target):
    tabla_frecuencia = data.groupby([variable, target]).size().unstack()
    total_buenos = tabla_frecuencia[0].sum()
    total_malos = tabla_frecuencia[1].sum()

    woe = {}
    iv = 0

    for categoria in tabla_frecuencia.index:
        evento_malo = tabla_frecuencia.loc[categoria, 1]
        evento_bueno = tabla_frecuencia.loc[categoria, 0]

        woe[categoria] = round((evento_malo / total_malos) / (evento_bueno / total_buenos), 2)
        iv += ((evento_malo / total_malos) - (evento_bueno / total_buenos)) * woe[categoria]

    return woe

# Lista de variables independientes para transformar
variables_independientes = ['edad','historial_crediticio','plazo_del_prestamo','tasa_de_interes',
'numero_de_dependientes','estado_laboral','proposito_del_prestamo','estado_civil','nivel_de_educacion']

# Transformar las variables independientes en WOE
for variable in variables_independientes:
    woe_dict = calcular_woe_variable(variable, 'resultado_del_riesgo')
    data[variable + '_WOE'] = data[variable].map(woe_dict)

In [7]:
data.head(3)

,id,edad,ingresos_anuales,historial_crediticio,monto_del_prestamo,plazo_del_prestamo,tasa_de_interes,deuda_existente,numero_de_dependientes,estado_laboral,...,periodo,edad_WOE,historial_crediticio_WOE,plazo_del_prestamo_WOE,tasa_de_interes_WOE,numero_de_dependientes_WOE,estado_laboral_WOE,proposito_del_prestamo_WOE,estado_civil_WOE,nivel_de_educacion_WOE
0,1,31,"$78,719.00",Bueno,"$35,098.00",30,11.8%,"$10,438.00",3,Autónomo,...,2022-06,0.99,1.00,1.28,1.19,1.13,1.0,1.11,1.05,0.97
1,2,29,"$43,660.00",Regular,"$43,304.00",34,7.4%,"$1,581.00",0,Desempleado,...,2022-05,1.32,1.03,0.93,0.72,1.06,1.0,1.07,1.05,0.97
2,3,58,"$56,278.00",Malo,"$12,629.00",9,10.9%,"$24,983.00",5,Desempleado,...,2022-04,1.12,0.97,1.03,1.70,0.98,1.0,0.96,0.96,1.01


In [8]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          10000 non-null  int64  
 1   edad                        10000 non-null  int64  
 2   ingresos_anuales            10000 non-null  object 
 3   historial_crediticio        10000 non-null  object 
 4   monto_del_prestamo          10000 non-null  object 
 5   plazo_del_prestamo          10000 non-null  int64  
 6   tasa_de_interes             10000 non-null  object 
 7   deuda_existente             10000 non-null  object 
 8   numero_de_dependientes      10000 non-null  int64  
 9   estado_laboral              10000 non-null  object 
 10  proposito_del_prestamo      10000 non-null  object 
 11  estado_civil                10000 non-null  object 
 12  nivel_de_educacion          10000 non-null  object 
 13  score_de_riesgo_interno     1000

### Calculo de iv

In [9]:
tabla_frecuencia = data.groupby([variable, 'resultado_del_riesgo']).size().unstack()
total_buenos = tabla_frecuencia[0].sum()
total_malos = tabla_frecuencia[1].sum()
# Crear una lista para almacenar los valores de IV por variable
iv_summary = []

# Calcular y mostrar el WOE y el IV para cada variable
for variable in variables_independientes:
    woe_dict = calcular_woe_variable(variable, 'resultado_del_riesgo')
    data[variable + '_WOE'] = data[variable].map(woe_dict)

    iv = 0
    for categoria, woe in woe_dict.items():
        evento_malo = data[data[variable] == categoria]['resultado_del_riesgo'].sum()
        evento_bueno = data[data[variable] == categoria]['resultado_del_riesgo'].count() - evento_malo

        iv += ((evento_malo / total_malos) - (evento_bueno / total_buenos)) * woe

    iv_summary.append({'Variable': variable, 'IV': iv})

# Crear un DataFrame para mostrar la tabla resumen de IV
iv_data = pd.DataFrame(iv_summary)

# Ordenar el DataFrame por IV en orden descendente
iv_data = iv_data.sort_values(by='IV', ascending=False)

# Mostrar la tabla resumen de IV
iv_data

,Variable,IV
3,tasa_de_interes,0.149392
0,edad,0.047804
2,plazo_del_prestamo,0.036587
6,proposito_del_prestamo,0.006170
4,numero_de_dependientes,0.004545
8,nivel_de_educacion,0.001737
7,estado_civil,0.001319
1,historial_crediticio,0.000601
5,estado_laboral,0.000018


In [10]:
# Filtrar las columnas que terminan con "_woe" más el target
data = data.filter(regex='_WOE$|resultado_del_riesgo|periodo')
data.head(3)

,resultado_del_riesgo,periodo,edad_WOE,historial_crediticio_WOE,plazo_del_prestamo_WOE,tasa_de_interes_WOE,numero_de_dependientes_WOE,estado_laboral_WOE,proposito_del_prestamo_WOE,estado_civil_WOE,nivel_de_educacion_WOE
0,1,2022-06,0.99,1.00,1.28,1.19,1.13,1.0,1.11,1.05,0.97
1,1,2022-05,1.32,1.03,0.93,0.72,1.06,1.0,1.07,1.05,0.97
2,1,2022-04,1.12,0.97,1.03,1.70,0.98,1.0,0.96,0.96,1.01


In [11]:
def get_na(data):
    qsna = data.shape[0] - data.isnull().sum(axis = 0) #Cantidad de NA
    qna = data.isnull().sum(axis = 0)
    ppna = round(100*(qna/data.shape[0]),2)
    aux = {'Datos sin Nas en qtd': qsna, 'Na en qtd': qna, 'Na en %': ppna}
    na = pd.DataFrame(data = aux)
    return na.sort_values(by = 'Na en qtd', ascending = False)

In [12]:
get_na(data)

,Datos sin Nas en qtd,Na en qtd,Na en %
resultado_del_riesgo,10000,0,0.0
periodo,10000,0,0.0
edad_WOE,10000,0,0.0
historial_crediticio_WOE,10000,0,0.0
plazo_del_prestamo_WOE,10000,0,0.0
tasa_de_interes_WOE,10000,0,0.0
numero_de_dependientes_WOE,10000,0,0.0
estado_laboral_WOE,10000,0,0.0
proposito_del_prestamo_WOE,10000,0,0.0
estado_civil_WOE,10000,0,0.0


In [13]:
# Guardar el conjunto de datos transformado en un nuevo archivo parquet
data.to_parquet('/Users/michael/Documents/Python/ProyestosPersonales/Modelos/statistical_models/xgboots/01 - Credit_risk_model/DATA/dataset_crediticio_Logict_woe.parquet', index=False)

### Logistic Regression

In [14]:
# Dividir los datos en conjunto de entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(train_data.shape)
print(test_data.shape)

(8000, 11)
(2000, 11)


In [15]:
# Tasa de malos conjunto de entrenamaiento
bad_rate_dev = train_data['resultado_del_riesgo'].value_counts().reset_index()
bad_rate_dev = bad_rate_dev.rename(columns={'index': 'Categorias', 'resultado_del_riesgo': 'Qtd'})
bad_rate_dev['%'] = round(bad_rate_dev['Qtd'] / bad_rate_dev['Qtd'].sum(), 1) * 100
bad_rate_dev

,Categorias,Qtd,%
0,1,7103,90.0
1,0,897,10.0


In [16]:
# Tasa de malos conjunto de validación
bad_rate_val = test_data['resultado_del_riesgo'].value_counts().reset_index()
bad_rate_val = bad_rate_val.rename(columns={'index': 'Categorias', 'resultado_del_riesgo': 'Qtd'})
bad_rate_val['%'] = round(bad_rate_val['Qtd'] / bad_rate_val['Qtd'].sum(), 1) * 100
bad_rate_val

,Categorias,Qtd,%
0,1,1789,90.0
1,0,211,10.0


In [17]:
# Definir las columnas predictoras (características) y la variable objetivo
features = ['edad_WOE','historial_crediticio_WOE','plazo_del_prestamo_WOE','tasa_de_interes_WOE',
'numero_de_dependientes_WOE','estado_laboral_WOE','proposito_del_prestamo_WOE','estado_civil_WOE',
'nivel_de_educacion_WOE']
target = 'resultado_del_riesgo'

In [18]:
# Crear el modelo de regresión logística
model = LogisticRegression()

# Entrenar el modelo en el conjunto de entrenamiento
model.fit(train_data[features], train_data[target])

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
# Evaluar el modelo en el conjunto de prueba
probabilities = model.predict_proba(test_data[features])[:, 1]
roc_auc = roc_auc_score(test_data[target], probabilities)

In [20]:
# Calcular las métricas de KS y Gini
fpr, tpr, _ = roc_curve(test_data[target], probabilities)
ks = max(tpr - fpr)
gini = 2 * roc_auc - 1

In [21]:
# Imprimir métricas
print("AUC-ROC:", roc_auc)
print("KS:", ks)
print("Gini:", gini)

AUC-ROC: 0.6037157033901224
KS: 0.167307320407228
Gini: 0.20743140678024474


In [22]:
# Calcular el score para cada cliente
train_data['score'] = np.round(model.predict_proba(train_data[features])[:, 1]*1000)

In [23]:
train_data.shape

(8000, 12)

### Tabla resumen distribuciones

In [24]:
# Dividir los datos en deciles naturales según el 'score'
train_data['decil'] = pd.qcut(train_data['score'], q=10, labels=False)

# Calcular las métricas por decil
summary_table = train_data.groupby('decil').agg({
    'score': ['min', 'max'],
    'resultado_del_riesgo': ['sum', lambda x: len(x) - sum(x)],
})

# Renombrar columnas
summary_table.columns = ['Min Probabilidad', 'Max Probabilidad', 'Eventos', 'No Eventos']

# Calcular tasas y acumulados
summary_table['Event Rate'] = summary_table['Eventos'] / (summary_table['Eventos'] + summary_table['No Eventos'])
summary_table['No Event Rate'] = 1 - summary_table['Event Rate']
summary_table['Cumulative Event Rate'] = summary_table['Eventos'].cumsum() / summary_table['Eventos'].sum()
summary_table['Cumulative No Event Rate'] = summary_table['No Eventos'].cumsum() / summary_table['No Eventos'].sum()

# Calcular el KS
summary_table['KS'] = np.abs(summary_table['Cumulative Event Rate'] - summary_table['Cumulative No Event Rate'])

# Calcular la columna Odds
summary_table['Odds'] = summary_table['No Eventos'] / summary_table['Eventos']

# Encontrar el índice del máximo KS
max_ks_index = summary_table['KS'].idxmax()

# Crear la función de formato para colores escalados
def format_color(value, min_value, max_value):
    normalized_value = (value - min_value) / (max_value - min_value)
    red = int(255 * (1 - normalized_value))
    green = int(255 * normalized_value)
    blue = 0
    return f'background-color: rgb({red}, {green}, {blue})'

min_ne_rate = summary_table['No Event Rate'].min()
max_ne_rate = summary_table['No Event Rate'].max()

# Crear la función de formato para porcentaje con dos decimales
def format_percent(value):
    return f'{value:.2%}'

# Crear la función de formato para valores enteros multiplicados por 1000
def format_int_thousands(value):
    return f'{int(value * 1):,.0f}'

# Crear la función de formato para valores con  dos decimales
def format_decimal(value):
    return f'{int(value * 100):.2f}'

# Crear la función de formato para valores con  dos decimales
def format_decimal_ks(value):
    return f'{value:.4f}'

# Aplicar el formato al DataFrame para las columnas "No Event Rate"
formatted_table = summary_table.style.applymap(lambda x: format_color(x, min_ne_rate, max_ne_rate), 
                                               subset=['No Event Rate'])
# Formato de columnas específicas
formatted_table = formatted_table.format({
    'Min Probabilidad': format_int_thousands,  # Sin decimales y multiplicado por 1000
    'Max Probabilidad': format_int_thousands,  # Sin decimales y multiplicado por 1000
    'Event Rate': format_percent,
    'No Event Rate': format_percent,
    'Cumulative Event Rate': format_percent,
    'Cumulative No Event Rate': format_percent,
    'KS': format_decimal_ks,
    'Odds': format_decimal
    })

# Crear la función de formato para resaltar el máximo KS en verde
def highlight_max_ks(s):
    is_max = s == s.max()
    return ['background-color: lightgreen' if v else '' for v in is_max]

# Resaltar el máximo KS en verde
formatted_table = formatted_table.apply(highlight_max_ks, subset=['KS'])

formatted_table


,Min Probabilidad,Max Probabilidad,Eventos,No Eventos,Event Rate,No Event Rate,Cumulative Event Rate,Cumulative No Event Rate,KS,Odds
decil,,,,,,,,,,
0,697,829,645,175,78.66%,21.34%,9.08%,19.51%,0.1043,27.00
1,830,850,654,127,83.74%,16.26%,18.29%,33.67%,0.1538,19.00
2,851,866,704,107,86.81%,13.19%,28.20%,45.60%,0.1740,15.00
3,867,879,691,100,87.36%,12.64%,37.93%,56.74%,0.1882,14.00
4,880,891,715,89,88.93%,11.07%,47.99%,66.67%,0.1867,12.00
5,892,902,718,79,90.09%,9.91%,58.10%,75.47%,0.1737,11.00
6,903,914,764,77,90.84%,9.16%,68.86%,84.06%,0.1520,10.00
7,915,927,733,57,92.78%,7.22%,79.18%,90.41%,0.1123,7.00
8,928,943,727,52,93.32%,6.68%,89.41%,96.21%,0.0680,7.00


In [ ]:
# Dividir los datos en deciles naturales según el 'score'
train_data['decil'] = pd.qcut(train_data['score'], q=10, labels=False)

# Calcular las métricas por decil
summary_table = train_data.groupby('decil').agg({
    'score': ['min', 'max'],
    'resultado_del_riesgo': ['sum', lambda x: len(x) - sum(x)],
})

# Renombrar columnas
summary_table.columns = ['Min Probabilidad', 'Max Probabilidad', 'Eventos', 'No Eventos']

# Calcular tasas y acumulados
summary_table['Event Rate'] = summary_table['Eventos'] / (summary_table['Eventos'] + summary_table['No Eventos'])
summary_table['No Event Rate'] = 1 - summary_table['Event Rate']
summary_table['Cumulative Event Rate'] = summary_table['Eventos'].cumsum() / summary_table['Eventos'].sum()
summary_table['Cumulative No Event Rate'] = summary_table['No Eventos'].cumsum() / summary_table['No Eventos'].sum()

# Calcular el KS
summary_table['KS'] = np.abs(summary_table['Cumulative Event Rate'] - summary_table['Cumulative No Event Rate'])

# Calcular la columna Odds
summary_table['Odds'] = summary_table['No Eventos'] / summary_table['Eventos']

# Encontrar el índice del máximo KS
max_ks_index = summary_table['KS'].idxmax()

# Crear la función de formato para colores escalados
def format_color(value, min_value, max_value):
    normalized_value = (value - min_value) / (max_value - min_value)
    green = int(255 * (1 - normalized_value))
    red = int(255 * normalized_value)
    blue = 0
    return f'background-color: rgb({red}, {green}, {blue})'

min_ne_rate = summary_table['No Event Rate'].min()
max_ne_rate = summary_table['No Event Rate'].max()

# Crear la función de formato para porcentaje con dos decimales
def format_percent(value):
    return f'{value:.2%}'

# Crear la función de formato para valores enteros multiplicados por 1000
def format_int_thousands(value):
    return f'{int(value * 1):,.0f}'

# Crear la función de formato para valores con dos decimales
def format_decimal(value):
    return f'{int(value * 100):.2f}'

# Crear la función de formato para valores con dos decimales
def format_decimal_ks(value):
    return f'{value:.4f}'

# Crear la función de formato para resaltar el máximo KS en verde
def highlight_max_ks(s):
    is_max = s == s.max()
    return ['background-color: lightgreen' if v else '' for v in is_max]

# Ordenar el DataFrame por la columna "decil" de forma descendente
summary_table = summary_table.sort_values(by='decil', ascending=False)

# Aplicar el formato al DataFrame para las columnas "No Event Rate"
formatted_table = summary_table.style.applymap(lambda x: format_color(x, min_ne_rate, max_ne_rate), 
                                               subset=['No Event Rate'])

# Formato de columnas específicas
formatted_table = formatted_table.format({
    'Min Probabilidad': format_int_thousands,  # Sin decimales y multiplicado por 1000
    'Max Probabilidad': format_int_thousands,  # Sin decimales y multiplicado por 1000
    'Event Rate': format_percent,
    'No Event Rate': format_percent,
    'Cumulative Event Rate': format_percent,
    'Cumulative No Event Rate': format_percent,
    'KS': format_decimal_ks,
    'Odds': format_decimal
    })

# Resaltar el máximo KS en verde
formatted_table = formatted_table.apply(highlight_max_ks, subset=['KS'])

# Mostrar la tabla
formatted_table


### Resumen gráfico

In [ ]:
# Graficar la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Calcular las métricas de KS en cada decil
grouped = train_data.groupby('decil', as_index=False).agg({'resultado_del_riesgo': 'mean', 'score': 'mean'})
ks_by_decil = np.abs(grouped['resultado_del_riesgo'] - grouped['score'])
ks_max = max(ks_by_decil)

# Graficar la curva KS
plt.figure(figsize=(8, 6))
plt.plot(grouped['decil'], ks_by_decil, marker='o', color='b')
plt.axhline(ks_max, color='r', linestyle='--', label='Max KS = {:.2f}'.format(ks_max))
plt.xlabel('decil')
plt.ylabel('KS Value')
plt.title('KS Curve')
plt.xticks(grouped['decil'])
plt.legend()
plt.show()